In [ ]:
# Imports
import locale
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()

# Importar e visualizar a base de dados
products_table = pd.read_excel("buscas.xlsx")
#display(tabela_produtos)

In [ ]:
# Funções
def data_treatment(product, banned):
    
    # Tratamento dos dados
    product = product.lower()
    product_terms = product.split(' ')
    banned = banned.lower()
    banned_terms = banned.split(' ')

    return product, product_terms, banned, banned_terms

def google_shopping_search(driver, product, banned, min_price, max_price):
    
    # Retorna os dados tratados
    product, product_terms, banned, banned_terms = data_treatment(product, banned)
    
    # Acessa o site do google
    driver.get("https://www.google.com/")

    # Acessa o campo de busca e faz a pesquisa
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(product, Keys.ENTER)

    # Acha e clica no elemento 'Shopping' do menu do google
    elements = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for element in elements:
        if "Shopping" in element.text:
            element.click()
            break

    # Pega cada resultado (ou div) do google shopping
    matches = driver.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    prices = [] 
    names = []
    links = []
    matches_list = []

    # Pega o preço, nome e link de cada um dos resultados
    for match in matches:
        price = match.find_element(By.CLASS_NAME, 'a8Pemb').text
        name = match.find_element(By.CLASS_NAME, 'Xjkr3b').text
        link = match.find_element(By.TAG_NAME, 'a').get_attribute('href')
        
        # Garante que não contenha valores vazios
        if price != '' and name != '' and link != '':
            
            # Trata os preços de string pra int usando o módulo 'locale'
            value = price
            value = value[2:]
            locale.setlocale(locale.LC_ALL, 'pt_BR.utf-8')
            value = locale.atof(value)
            value = float(value)
            
            ban = False
            # Verifica se o produto encontrado contém algum dos termos banidos
            for term in banned_terms:
                if term in name.lower():
                    ban = True
                    break
            
            # Verifica se o produto encontrado possui as características necessárias
            for term in product_terms:
                if not term in name.lower():
                    ban = True
                    break
            
            if min_price <= value <= max_price and ban == False:
                prices.append(price)
                names.append(name)
                links.append(link)
                matches_list.append((price, name, link))
    
    # Retorna uma lista de tuplas pra cada resultado
    return matches_list

def buscape_search(driver, product, banned, min_price, max_price):
    
    # Retorna os dados tratados
    product, product_terms, banned, banned_terms = data_treatment(product, banned)
    
    # Acessa o site do buscapé
    driver.get("https://www.buscape.com.br/")

    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[1]/input').send_keys(product, Keys.ENTER)
    
    time.sleep(10)

    matches = driver.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')

    prices = [] 
    names = []
    links = []
    matches_list = []

    for match in matches:
        price = match.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text
        name = match.get_attribute('title')
        link = match.get_attribute('href')

        # Garante que não contenha valores vazios
        if price != '' and name != '' and link != '':
            
            # Trata os preços de string pra int usando o módulo 'locale'
            value = price
            value = value[2:]
            locale.setlocale(locale.LC_ALL, 'pt_BR.utf-8')
            value = locale.atof(value)
            value = float(value)
            
            ban = False
            # Verifica se o produto encontrado contém algum dos termos banidos
            for term in banned_terms:
                if term in name.lower():
                    ban = True
                    break
            
            # Verifica se o produto encontrado possui as características necessárias
            for term in product_terms:
                if not term in name.lower():
                    ban = True
                    break
            
            if min_price <= value <= max_price and ban == False:
                prices.append(price)
                names.append(name)
                links.append(link)
                matches_list.append((price, name, link))

    # Retorna uma lista de tuplas pra cada resultado
    return matches_list

In [ ]:
product = 'Iphone 12 64 gb vermelho'
banned = 'watch mini case charger usado seminovo semi-novo 11 X XR XS' 
min_price = 3000
max_price = 5000

print(google_shopping_search(driver, product, banned, min_price, max_price))
print(buscape_search(driver, product, banned, min_price, max_price))